In [16]:
import torch
torch.manual_seed(1)    # reproducible

# 假数据
x = torch.unsqueeze(torch.linspace(-1, 1, 100), dim=1)  # x data (tensor), shape=(100, 1)
y = x.pow(2)# + 0.2*torch.rand(x.size())  # noisy y data (tensor), shape=(100, 1)

def save():
    # 建网络
    net1 = torch.nn.Sequential(
        torch.nn.Linear(1, 10),
        torch.nn.ReLU(),
        torch.nn.Linear(10, 1)
    )
    optimizer = torch.optim.SGD(net1.parameters(), lr=0.5)
    loss_func = torch.nn.MSELoss()

    # 训练
    for t in range(200):
        prediction = net1(x)
        loss = loss_func(prediction, y)
        print(loss)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    torch.save(net1.state_dict(), 'net_params.pkl')   # 只保存网络中的参数 (速度快, 占内存少)
def restore_params():
    # 新建 net3
    net3 = torch.nn.Sequential(
        torch.nn.Linear(1, 10),
        torch.nn.ReLU(),
        torch.nn.Linear(10, 1)
    )

    # 将保存的参数复制到 net3
    net3.load_state_dict(torch.load('net_params.pkl'))
    prediction = net3(x)
#torch.save(net1, 'net.pkl')  # 保存整个网络
#torch.save(net1.state_dict(), 'net_params.pkl')   # 只保存网络中的参数 (速度快, 占内存少)

# 保存 net1 (1. 整个网络, 2. 只有参数)
save()
# 提取网络参数, 复制到新网络
#restore_params()
net3 = torch.nn.Sequential(
        torch.nn.Linear(1, 10),
        torch.nn.ReLU(),
        torch.nn.Linear(10, 1)
    )

# 将保存的参数复制到 net3
net3.load_state_dict(torch.load('net_params.pkl'))
prediction = net3(x)
torch.nn.MSELoss()(prediction,y)

tensor(0.3642, grad_fn=<MseLossBackward>)
tensor(0.5271, grad_fn=<MseLossBackward>)
tensor(0.5309, grad_fn=<MseLossBackward>)
tensor(0.2504, grad_fn=<MseLossBackward>)
tensor(0.1006, grad_fn=<MseLossBackward>)
tensor(0.0689, grad_fn=<MseLossBackward>)
tensor(0.0617, grad_fn=<MseLossBackward>)
tensor(0.0572, grad_fn=<MseLossBackward>)
tensor(0.0532, grad_fn=<MseLossBackward>)
tensor(0.0494, grad_fn=<MseLossBackward>)
tensor(0.0457, grad_fn=<MseLossBackward>)
tensor(0.0423, grad_fn=<MseLossBackward>)
tensor(0.0391, grad_fn=<MseLossBackward>)
tensor(0.0360, grad_fn=<MseLossBackward>)
tensor(0.0332, grad_fn=<MseLossBackward>)
tensor(0.0305, grad_fn=<MseLossBackward>)
tensor(0.0280, grad_fn=<MseLossBackward>)
tensor(0.0257, grad_fn=<MseLossBackward>)
tensor(0.0235, grad_fn=<MseLossBackward>)
tensor(0.0216, grad_fn=<MseLossBackward>)
tensor(0.0198, grad_fn=<MseLossBackward>)
tensor(0.0181, grad_fn=<MseLossBackward>)
tensor(0.0166, grad_fn=<MseLossBackward>)
tensor(0.0153, grad_fn=<MseLossBac

In [58]:
!pwd

/root/server/Semantic-Loss/complex_constraints/curriculum learning


In [26]:
import sys
sys.path.insert(0,'/root/server/Semantic-Loss/complex_constraints/curriculum learning/craft-env')
import numpy as np
import time

import env_factory

#https://github.com/Feryal/craft-env/blob/master/env.py

visualise = True
recipes_path = "/root/server/Semantic-Loss/complex_constraints/curriculum learning/craft-env/resources/recipes.yaml"
hints_path = "/root/server/Semantic-Loss/complex_constraints/curriculum learning/craft-env/resources/hints.yaml"
env_sampler = env_factory.EnvironmentFactory(
  recipes_path, hints_path, max_steps=100, reuse_environments=False,
  visualise=visualise)

env = env_sampler.sample_environment(task_name='get[rock]')
print("Environment: task {}: {}".format(env.task_name, env.task))


n_steps=100 * 3
possible_actions = env.action_specs()

observations = env.reset()
for t in range(0):
    # Random action
    action = np.random.choice(list(possible_actions.values()))
    
    # Step (this will plot if visualise is True)
    observations,reward, done, _ = env.step(action)
    print(t,reward)
    if visualise:
        env.render_matplotlib(frame=observations['image'])
    else:
        print("[{}] reward={} done={} \n observations: {}".format(
          t, reward, done, observations))

    if reward:
        rewarding_frame = observations['image'].copy()
        rewarding_frame[:40] *= np.array([0, 1, 0])
        env.render_matplotlib(frame=rewarding_frame, delta_time=0.7)
        print("[{}] Got a rewaaaard! {:.1f}".format(t, reward))
    elif done:
        env.render_matplotlib(
          frame=np.zeros_like(observations['image']), delta_time=0.3)
        print("[{}] Finished with nothing... Reset".format(t))

#https://github.com/Feryal/craft-env/blob/master/env.py

Environment: task get[rock]: Task(goal=(1, 10), steps=(5,))


In [36]:
import gym
import math
import random
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
from collections import namedtuple
from itertools import count
from PIL import Image

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torchvision.transforms as T

# set up matplotlib
is_ipython = 'inline' in matplotlib.get_backend()
if is_ipython:
    from IPython import display

plt.ion()

# if gpu is to be used
device = torch.device("cpu")
Transition = namedtuple('Transition',
                        ('state', 'action', 'next_state', 'reward'))


class ReplayMemory(object):

    def __init__(self, capacity):
        self.capacity = capacity
        self.memory = []
        self.position = 0

    def push(self, *args):
        """Saves a transition."""
        if len(self.memory) < self.capacity:
            self.memory.append(None)
        self.memory[self.position] = Transition(*args)
        self.position = (self.position + 1) % self.capacity

    def sample(self, batch_size):
        return random.sample(self.memory, batch_size)

    def __len__(self):
        return len(self.memory)
class DQN(nn.Module):

    def __init__(self):
        super(DQN, self).__init__()
        self.l1 = nn.Linear(1586,2048)
        self.l2 = nn.Linear(2048,3072)
        self.l3 = nn.Linear(3072,4)

    # Called with either one element to determine next action, or a batch
    # during optimization. Returns tensor([[left0exp,right0exp]...]).
    def forward(self, x):
        x = F.relu(self.l1(x))
        x = F.relu(self.l2(x))
        x = F.relu(self.l3(x))
        return x
resize = T.Compose([T.ToPILImage(),
                    T.Resize(40, interpolation=Image.CUBIC),
                    T.ToTensor()])


def get_cart_location(screen_width):
    world_width = env.x_threshold * 2
    scale = screen_width / world_width
    return int(env.state[0] * scale + screen_width / 2.0)  # MIDDLE OF CART



env.reset()
BATCH_SIZE = 128
GAMMA = 0.999
EPS_START = 0.9
EPS_END = 0.05
EPS_DECAY = 200
TARGET_UPDATE = 10

# Get screen size so that we can initialize layers correctly based on shape
# returned from AI gym. Typical dimensions at this point are close to 3x40x90
# which is the result of a clamped and down-scaled render buffer in get_screen()

# Get number of actions from gym action space
n_actions = 4

policy_net = DQN()#.to(device)
target_net = DQN()#.to(device)
target_net.load_state_dict(policy_net.state_dict())
target_net.eval()

optimizer = optim.RMSprop(policy_net.parameters())
memory = ReplayMemory(10000)


steps_done = 0


def select_action(state):
    global steps_done
    sample = random.random()
    eps_threshold = EPS_END + (EPS_START - EPS_END) * \
        math.exp(-1. * steps_done / EPS_DECAY)
    steps_done += 1
    if sample > 0.9:
        with torch.no_grad():
            # t.max(1) will return largest column value of each row.
            # second column on max result is index of where max element was
            # found, so we pick action with the larger expected reward.
            return policy_net(state).argmax().view(1, 1)
    else:
        return torch.tensor([[random.randrange(n_actions)]])
def select_action2(state):
    return policy_net(state).argmax().view(1, 1)


episode_durations = []


In [37]:
def optimize_model():
    if len(memory) < BATCH_SIZE:
        return
    transitions = memory.sample(BATCH_SIZE)
    # Transpose the batch (see https://stackoverflow.com/a/19343/3343043 for
    # detailed explanation). This converts batch-array of Transitions
    # to Transition of batch-arrays.
    batch = Transition(*zip(*transitions))
    # Compute a mask of non-final states and concatenate the batch elements
    # (a final state would've been the one after which simulation ended)
    non_final_mask = torch.tensor(tuple(map(lambda s: s is not None,
                                          batch.next_state)), dtype=torch.bool)
    tmp=[s for s in batch.next_state if s is not None]
 #   print('tmp:',tmp[0])
  #  print('len(tmp):',len(tmp))
    non_final_next_states = torch.cat(tmp).view([len(tmp),len(batch.state[0])])
    state_batch = torch.cat(batch.state).view([BATCH_SIZE,len(batch.state[0])])
    action_batch = torch.cat(batch.action)
    reward_batch = torch.cat(batch.reward)

    # Compute Q(s_t, a) - the model computes Q(s_t), then we select the
    # columns of actions taken. These are the actions which would've been taken
    # for each batch state according to policy_net
    state_action_values = policy_net(state_batch).gather(1, action_batch)

    # Compute V(s_{t+1}) for all next states.
    # Expected values of actions for non_final_next_states are computed based
    # on the "older" target_net; selecting their best reward with max(1)[0].
    # This is merged based on the mask, such that we'll have either the expected
    # state value or 0 in case the state was final.
    next_state_values = torch.zeros(BATCH_SIZE)
    next_state_values[non_final_mask] = target_net(non_final_next_states).max(1)[0].detach()
    # Compute the expected Q values
    expected_state_action_values = (next_state_values * GAMMA) + reward_batch

    # Compute Huber loss
    loss = F.smooth_l1_loss(state_action_values, expected_state_action_values.unsqueeze(1))

    # Optimize the model
    optimizer.zero_grad()
    loss.backward()
    for param in policy_net.parameters():
        param.grad.data.clamp_(-1, 1)
    optimizer.step()
num_episodes = 50
for i_episode in range(num_episodes):
    # Initialize the environment and state
    print('--------------4:',i_episode)
    state=torch.tensor(list( env.reset()['features']))
    
    for t in count():
        # Select and perform an action
        action = select_action(state)
        print(action)
        observations,reward, done, _= env.step(action.item())
        print('hhhhhhhhhhhhh',reward,done)
        reward = torch.tensor([reward])
        # Observe new state
        if not done:
            next_state =torch.tensor(list( observations['features']))
        else:
            next_state = None

        # Store the transition in memory
        memory.push(state, action, next_state, reward)

        # Move to the next state
        state = next_state

        # Perform one step of the optimization (on the target network)
        optimize_model()
        if done:
            episode_durations.append(t + 1)
            break
    # Update the target network, copying all weights and biases in DQN
    if i_episode % TARGET_UPDATE == 0:
        target_net.load_state_dict(policy_net.state_dict())

print('Complete')

--------------4: 0
tensor([[3]])
hhhhhhhhhhhhh 0.0 False
tensor([[2]])
hhhhhhhhhhhhh 0.0 False
tensor([[2]])
hhhhhhhhhhhhh 0.0 False
tensor([[1]])
hhhhhhhhhhhhh 0.0 False
tensor([[3]])
hhhhhhhhhhhhh 0.0 False
tensor([[0]])
hhhhhhhhhhhhh 0.0 False
tensor([[0]])
hhhhhhhhhhhhh 0.0 False
tensor([[0]])
hhhhhhhhhhhhh 0.0 False
tensor([[0]])
hhhhhhhhhhhhh 0.0 False
tensor([[3]])
hhhhhhhhhhhhh 0.0 False
tensor([[0]])
hhhhhhhhhhhhh 0.0 False
tensor([[0]])
hhhhhhhhhhhhh 0.0 False
tensor([[0]])
hhhhhhhhhhhhh 0.0 False
tensor([[1]])
hhhhhhhhhhhhh 0.0 False
tensor([[3]])
hhhhhhhhhhhhh 0.0 False
tensor([[2]])
hhhhhhhhhhhhh 0.0 False
tensor([[1]])
hhhhhhhhhhhhh 0.0 False
tensor([[3]])
hhhhhhhhhhhhh 0.0 False
tensor([[1]])
hhhhhhhhhhhhh 0.0 False
tensor([[1]])
hhhhhhhhhhhhh 0.0 False
tensor([[0]])
hhhhhhhhhhhhh 0.0 False
tensor([[0]])
hhhhhhhhhhhhh 0.0 False
tensor([[0]])
hhhhhhhhhhhhh 0.0 False
tensor([[1]])
hhhhhhhhhhhhh 0.0 False
tensor([[2]])
hhhhhhhhhhhhh 0.0 False
tensor([[0]])
hhhhhhhhhhhhh 0.0

tensor([[0]])
hhhhhhhhhhhhh 0.0 False
tensor([[1]])
hhhhhhhhhhhhh 0.0 False
tensor([[0]])
hhhhhhhhhhhhh 0.0 False
tensor([[2]])
hhhhhhhhhhhhh 0.0 False
tensor([[2]])
hhhhhhhhhhhhh 0.0 False
tensor([[0]])
hhhhhhhhhhhhh 0.0 False
tensor([[3]])
hhhhhhhhhhhhh 0.0 False
tensor([[0]])
hhhhhhhhhhhhh 0.0 False
tensor([[1]])
hhhhhhhhhhhhh 0.0 False
tensor([[0]])
hhhhhhhhhhhhh 0.0 False
tensor([[2]])
hhhhhhhhhhhhh 0.0 False
tensor([[0]])
hhhhhhhhhhhhh 0.0 False
tensor([[2]])
hhhhhhhhhhhhh 0.0 False
tensor([[0]])
hhhhhhhhhhhhh 0.0 False
tensor([[1]])
hhhhhhhhhhhhh 0.0 False
tensor([[0]])
hhhhhhhhhhhhh 0.0 False
tensor([[0]])
hhhhhhhhhhhhh 0.0 False
tensor([[0]])
hhhhhhhhhhhhh 0.0 False
tensor([[1]])
hhhhhhhhhhhhh 0.0 False
tensor([[1]])
hhhhhhhhhhhhh 0.0 False
tensor([[0]])
hhhhhhhhhhhhh 0.0 False
tensor([[2]])
hhhhhhhhhhhhh 0.0 False
tensor([[1]])
hhhhhhhhhhhhh 0.0 False
tensor([[0]])
hhhhhhhhhhhhh 0.0 False
tensor([[1]])
hhhhhhhhhhhhh 0.0 False
tensor([[2]])
hhhhhhhhhhhhh 0.0 False
tensor([[3]]

tensor([[3]])
hhhhhhhhhhhhh 0.0 False
tensor([[0]])
hhhhhhhhhhhhh 0.0 False
tensor([[0]])
hhhhhhhhhhhhh 0.0 False
tensor([[0]])
hhhhhhhhhhhhh 0.0 False
tensor([[3]])
hhhhhhhhhhhhh 0.0 False
tensor([[1]])
hhhhhhhhhhhhh 0.0 False
tensor([[0]])
hhhhhhhhhhhhh 0.0 False
tensor([[0]])
hhhhhhhhhhhhh 0.0 False
tensor([[0]])
hhhhhhhhhhhhh 0.0 False
tensor([[1]])
hhhhhhhhhhhhh 0.0 False
tensor([[2]])
hhhhhhhhhhhhh 0.0 False
tensor([[1]])
hhhhhhhhhhhhh 0.0 False
tensor([[0]])
hhhhhhhhhhhhh 0.0 False
tensor([[2]])
hhhhhhhhhhhhh 0.0 False
tensor([[3]])
hhhhhhhhhhhhh 0.0 False
tensor([[2]])
hhhhhhhhhhhhh 0.0 False
tensor([[2]])
hhhhhhhhhhhhh 0.0 False
tensor([[1]])
hhhhhhhhhhhhh 0.0 False
tensor([[1]])
hhhhhhhhhhhhh 0.0 False
tensor([[1]])
hhhhhhhhhhhhh 0.0 False
tensor([[2]])
hhhhhhhhhhhhh 0.0 False
tensor([[0]])
hhhhhhhhhhhhh 0.0 False
tensor([[3]])
hhhhhhhhhhhhh 0.0 False
tensor([[2]])
hhhhhhhhhhhhh 0.0 False
tensor([[2]])
hhhhhhhhhhhhh 0.0 False
tensor([[0]])
hhhhhhhhhhhhh 0.0 False
tensor([[3]]

tensor([[0]])
hhhhhhhhhhhhh 0.0 False
tensor([[2]])
hhhhhhhhhhhhh 0.0 False
tensor([[1]])
hhhhhhhhhhhhh 0.0 False
tensor([[0]])
hhhhhhhhhhhhh 0.0 False
tensor([[2]])
hhhhhhhhhhhhh 0.0 False
tensor([[1]])
hhhhhhhhhhhhh 0.0 False
tensor([[0]])
hhhhhhhhhhhhh 0.0 False
tensor([[1]])
hhhhhhhhhhhhh 0.0 False
tensor([[2]])
hhhhhhhhhhhhh 0.0 False
tensor([[2]])
hhhhhhhhhhhhh 0.0 False
tensor([[1]])
hhhhhhhhhhhhh 0.0 False
tensor([[2]])
hhhhhhhhhhhhh 0.0 False
tensor([[1]])
hhhhhhhhhhhhh 0.0 False
tensor([[3]])
hhhhhhhhhhhhh 0.0 False
tensor([[1]])
hhhhhhhhhhhhh 0.0 False
tensor([[0]])
hhhhhhhhhhhhh 0.0 False
tensor([[0]])
hhhhhhhhhhhhh 0.0 False
tensor([[1]])
hhhhhhhhhhhhh 0.0 False
tensor([[0]])
hhhhhhhhhhhhh 0.0 False
tensor([[3]])
hhhhhhhhhhhhh 0.0 False
tensor([[2]])
hhhhhhhhhhhhh 0.0 False
tensor([[3]])
hhhhhhhhhhhhh 0.0 False
tensor([[2]])
hhhhhhhhhhhhh 0.0 False
tensor([[2]])
hhhhhhhhhhhhh 0.0 False
tensor([[0]])
hhhhhhhhhhhhh 0.0 False
tensor([[2]])
hhhhhhhhhhhhh 0.0 False
tensor([[1]]

tensor([[2]])
hhhhhhhhhhhhh 0.0 False
tensor([[2]])
hhhhhhhhhhhhh 0.0 False
tensor([[3]])
hhhhhhhhhhhhh 0.0 False
tensor([[1]])
hhhhhhhhhhhhh 0.0 False
tensor([[1]])
hhhhhhhhhhhhh 0.0 False
tensor([[0]])
hhhhhhhhhhhhh 0.0 False
tensor([[2]])
hhhhhhhhhhhhh 0.0 False
tensor([[2]])
hhhhhhhhhhhhh 0.0 False
tensor([[0]])
hhhhhhhhhhhhh 0.0 False
tensor([[3]])
hhhhhhhhhhhhh 0.0 False
tensor([[3]])
hhhhhhhhhhhhh 0.0 False
tensor([[2]])
hhhhhhhhhhhhh 0.0 False
tensor([[0]])
hhhhhhhhhhhhh 0.0 False
tensor([[0]])
hhhhhhhhhhhhh 0.0 False
tensor([[0]])
hhhhhhhhhhhhh 0.0 False
tensor([[1]])
hhhhhhhhhhhhh 0.0 False
tensor([[3]])
hhhhhhhhhhhhh 0.0 False
tensor([[2]])
hhhhhhhhhhhhh 0.0 False
tensor([[0]])
hhhhhhhhhhhhh 0.0 False
tensor([[0]])
hhhhhhhhhhhhh 0.0 False
tensor([[0]])
hhhhhhhhhhhhh 0.0 False
tensor([[0]])
hhhhhhhhhhhhh 0.0 False
tensor([[0]])
hhhhhhhhhhhhh 0.0 False
tensor([[2]])
hhhhhhhhhhhhh 0.0 False
tensor([[3]])
hhhhhhhhhhhhh 0.0 False
tensor([[0]])
hhhhhhhhhhhhh 0.0 False
tensor([[1]]

tensor([[0]])
hhhhhhhhhhhhh 0.0 False
tensor([[2]])
hhhhhhhhhhhhh 0.0 False
tensor([[3]])
hhhhhhhhhhhhh 0.0 False
tensor([[3]])
hhhhhhhhhhhhh 0.0 False
tensor([[0]])
hhhhhhhhhhhhh 0.0 False
tensor([[2]])
hhhhhhhhhhhhh 0.0 False
tensor([[0]])
hhhhhhhhhhhhh 0.0 False
tensor([[2]])
hhhhhhhhhhhhh 0.0 False
tensor([[1]])
hhhhhhhhhhhhh 0.0 False
tensor([[2]])
hhhhhhhhhhhhh 0.0 False
tensor([[0]])
hhhhhhhhhhhhh 0.0 False
tensor([[2]])
hhhhhhhhhhhhh 0.0 False
tensor([[1]])
hhhhhhhhhhhhh 0.0 False
tensor([[3]])
hhhhhhhhhhhhh 0.0 False
tensor([[0]])
hhhhhhhhhhhhh 0.0 False
tensor([[0]])
hhhhhhhhhhhhh 0.0 False
tensor([[0]])
hhhhhhhhhhhhh 0.0 False
tensor([[0]])
hhhhhhhhhhhhh 0.0 False
tensor([[0]])
hhhhhhhhhhhhh 0.0 False
tensor([[3]])
hhhhhhhhhhhhh 0.0 False
tensor([[2]])
hhhhhhhhhhhhh 0.0 False
tensor([[1]])
hhhhhhhhhhhhh 0.0 True
--------------4: 11
tensor([[0]])
hhhhhhhhhhhhh 0.0 False
tensor([[1]])
hhhhhhhhhhhhh 0.0 False
tensor([[2]])
hhhhhhhhhhhhh 0.0 False
tensor([[3]])
hhhhhhhhhhhhh 0.0

KeyboardInterrupt: 

In [1]:

complexity=[]
envs=[]
for i in range(21):
    envs.append(env_sampler.sample_environment(task_name=list(env_sampler.hints.keys())[i]))
    complexity.append(len(list(env_sampler.hints.values())[i])**1)
eps=0.1
vdict={}
for i in range(21):
    for j in range(21):
        vdict[(i,j)]=(len(set(list(env_sampler.hints.values())[i])&set(list(env_sampler.hints.values())[j]))+eps)/(complexity[i])/(1+complexity[j]-complexity[i]+eps)
taskIndex=[]
for i in range(20):
    if (vdict[(i,20)])>0.1:
        taskIndex.append(i)
links=[]
degreeout={}
for i in taskIndex:
    degreeout[i]=0
for i in taskIndex:
    tmp={}
    for j in taskIndex[:-1]:
        if i!=j:
            tmp[vdict[(j,i)]]=j
    degreeout[tmp[max(list(tmp.keys()))]]+=1
    links.append((tmp[max(list(tmp.keys()))],i))
for i in taskIndex[:-1]:
    if degreeout[i]==0:
        links.append((i,taskIndex[-1]))
        degreeout[i]=1
degreein={}
for i in taskIndex:
    degreein[i]=0
for (i,j) in links:
    degreein[j]+=1
import networkx as nx
G = nx.Graph() #定义了一个空图
for i in taskIndex:
    G.add_node(i) #这个图中增加了1节点
G.add_edges_from(links) 
#也可以
import matplotlib.pyplot as plt
import networkx as nx

nx.draw(G, with_labels=True)
plt.show()

/root/server/Semantic-Loss/complex_constraints/curriculum learning/craft-env/cookbook.py:12: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  recipes = yaml.load(recipes_f)
/root/server/Semantic-Loss/complex_constraints/curriculum learning/craft-env/env_factory.py:43: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  self.hints = yaml.load(hints_f)


Environment: task get[rock]: Task(goal=(1, 10), steps=(5,))


<Figure size 640x480 with 1 Axes>

In [1]:
import sys
sys.path.insert(0,'/root/server/Semantic-Loss/complex_constraints/curriculum learning/craft-env')
import numpy as np
import time

import env_factory

#https://github.com/Feryal/craft-env/blob/master/env.py

visualise = True
recipes_path = "/root/server/Semantic-Loss/complex_constraints/curriculum learning/craft-env/resources/recipes.yaml"
hints_path = "/root/server/Semantic-Loss/complex_constraints/curriculum learning/craft-env/resources/hints.yaml"
env_sampler = env_factory.EnvironmentFactory(
  recipes_path, hints_path, max_steps=100, reuse_environments=False,
  visualise=visualise)

env = env_sampler.sample_environment(task_name='get[rock]')
print("Environment: task {}: {}".format(env.task_name, env.task))


n_steps=100 * 3
possible_actions = env.action_specs()

observations = env.reset()
complexity=[]
envs=[]
for i in range(21):
    envs.append(env_sampler.sample_environment(task_name=list(env_sampler.hints.keys())[i]))
    complexity.append(len(list(env_sampler.hints.values())[i])**1)
eps=0.1
similaritydict={}
for i in range(21):
    for j in range(21):
        similaritydict[(i,j)]=len(set(list(env_sampler.hints.values())[i])&set(list(env_sampler.hints.values())[j]))/complexity[i]          
complexityRankDict={}
for i in set(complexity):
    complexityRankDict[i]=[]
for i in range(21):
    complexityRankDict[complexity[i]].append(i)

Environment: task get[rock]: Task(goal=(1, 10), steps=(5,))


/root/server/Semantic-Loss/complex_constraints/curriculum learning/craft-env/cookbook.py:12: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  recipes = yaml.load(recipes_f)
/root/server/Semantic-Loss/complex_constraints/curriculum learning/craft-env/env_factory.py:43: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  self.hints = yaml.load(hints_f)
